# <center>"ВИКИШОП"</center>
___
___

# <center>Самостоятельные проект по теме "Машинное обучение для текстов"</center>
___
___

# <center>Описание проекта</center>   

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества F1 не меньше 0.75.   

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.
___

# <center>Состав проекта<center> <a id = 'back'></a>

[1. Загрузка и подготовка данных](#1)   
    
[2. Обучение моделей](#2)   
    
[3. Выводы](#3)   
___

# <center>Описание данных</center>

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.
___

Импорт необходимых библиотек

In [1]:
import pandas as pd
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import re
import spacy
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Anton\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Anton\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Anton\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


<a id = '1'></a>
# 1. Загрузка и подготовка данных   
___

Считываем данные и смотрим основную информацию.

In [2]:
toxic_comments = pd.read_csv('toxic_comments.csv')
toxic_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Посмотрим текст комментариев.

In [3]:
toxic_comments.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Вычленим из датасета текстовую колонку в отдельный корпус. Это необходимо для предобработки, в дальнейшем мы вернем стобец признаков при разбиении на выборки.

In [4]:
corpus = list(toxic_comments['text'])

После просмотра нескольких случайных выборок видим следующее:   
- текст на английском языке;   
- текст требует лемматизации;   
- текст требует токенизации.   

Займемся лемматизацией корпуса.

Напишем функцию очистки текста от лишних символов с помощью регулярных выражений.

In [5]:
def clear_text(text):
    temp = re.sub(r'[^a-zA-Z\']', ' ', text)
    return " ".join(temp.split())

Проверим работоспособность написанных функций на примере одного (или нескольких) комментария.

In [6]:
clear_lemm_corp = []

In [7]:
nlp = spacy.load("en_core_web_sm")

In [8]:
doc = nlp(clear_text(corpus[0]))

In [9]:
" ".join([token.lemma_ for token in doc])

'explanation why the edit make under -PRON- username Hardcore Metallica Fan be revert -PRON- be not vandalism just closure on some gas after -PRON- vote at New York Dolls FAC and please do not remove the template from the talk page since -PRON- be retire now'

In [10]:
for i in range(len(corpus)):
    corpus[i] = corpus[i].lower()

In [11]:
%%time
for i in range(len(corpus)):
    doc = nlp(clear_text(corpus[i]))
    clear_lemm_corp.append(" ".join([token.lemma_ for token in doc]))

Wall time: 33min 50s


In [12]:
print(clear_lemm_corp[777])

gift card for download re as of december -PRON- be not possible to use gift card to purchase mp s even though the gift card faq claim yes amazon com gift card can be use to buy amazon mp and unbox download strictly speak this be a false statement as be able to pay for order with a gift card at that time one person already remove that line but -PRON- be revert be there a good way to go about provide verification


Объединим лемматизированный и очищенный текст с целевым признаком. Проверим результат объединения.

In [13]:
full_data = pd.DataFrame(clear_lemm_corp, columns = ['lemm_text'])
full_data['toxic'] = toxic_comments['toxic']

In [14]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   lemm_text  159571 non-null  object
 1   toxic      159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [15]:
print(full_data.sample(10))

                                                lemm_text  toxic
44574   screw -PRON- why do not -PRON- wikipedia jerk ...      1
42490   -PRON- moron do not accuse -PRON- to go and st...      1
117616  this article be a piece of shit read -PRON- an...      1
74530   about the name of chinese general in zerohour ...      0
63469   inquiry on notability hi drmie in order not to...      0
142468  please delete all duplication below and merge ...      0
28786   interesting remark be make about this article ...      0
105322                            redirect talk trs model      0
100178  fuck -PRON- fuck misterwiki -PRON- fuck mister...      1
56076   yearbook photo -PRON- be sit on the complete s...      0


Разобьем данные на выборки. На тестовую и валидационную отведем по 20% исходных данных.

In [16]:
train, temp = train_test_split(full_data, random_state = 481516, test_size=0.4)
valid, test = train_test_split(temp, random_state = 481516, test_size=0.5)
print(train.shape, valid.shape, test.shape)

(95742, 2) (31914, 2) (31915, 2)


In [17]:
target_train = train['toxic']
corpus_train = train['lemm_text'].values.astype('U')
target_valid = valid['toxic']
corpus_valid = valid['lemm_text'].values.astype('U')
target_test = test['toxic']
corpus_test = test['lemm_text'].values.astype('U')

In [18]:
print(corpus_train[10])

opinion -PRON- be a simple opinion -PRON- be entitle to disagree with -PRON- and or respond but a blocking be uncalled for i do not swear at all and i believe i be calm please block aolanaonwaswronglyaccuse along with calton for -PRON- insult against -PRON- if -PRON- comment warrant block


In [20]:
print(corpus_train.shape, corpus_valid.shape, corpus_test.shape)

(95742,) (31914,) (31915,)


In [21]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words = stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Anton\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
tf_idf_train = count_tf_idf.fit_transform(corpus_train)
tf_idf_valid = count_tf_idf.transform(corpus_valid)
tf_idf_test = count_tf_idf.transform(corpus_test)

In [23]:
print(tf_idf_train.shape, tf_idf_valid.shape, tf_idf_test.shape)

(95742, 113376) (31914, 113376) (31915, 113376)


[*к содержанию*](#back)
___

<a id = '2'></a>
# 2. Обучение моделей
___

В качестве обучаемых моделей были испробованы: линейная регрессия, логистическая регрессия и случайный лес.   
Далее в этом разделе, для экономии времени, оставлена реализация только логистической регрессии, причины будут отражены в выводых.

In [24]:
logr_model = LogisticRegression(random_state = 481516, class_weight = 'balanced', C = 5.0)

In [25]:
%%time
logr_model.fit(tf_idf_train, target_train)

Wall time: 3.79 s


C:\Users\Anton\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=5.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=481516, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [26]:
logr_pred = logr_model.predict(tf_idf_valid)

print(f1_score(target_valid, logr_pred))

0.763206606863164


In [27]:
logr_pred = logr_model.predict(tf_idf_test)

print(f1_score(target_test, logr_pred))

0.7617568140093206


[*к содержанию*](#back)
___

<a id = '3'></a>
# 3. Выводы
___

- Нам были предоставлены данные с комментариями магазина "Викишоп";   
- Исходные данные были очищены от ненужных символов, лемматизированы и преобразованы в векторное представление;   
- Для лемматизации были испробованы `WordNetLemmatizer` и `spacy`. Последний показал себя значительно лучше для поставленной задачи: настройка параметров лемматизации более простая, в то время как время лемматизации значительно ниже (примерно 30 минут против нескольких часов при использовании WordNet);    
- Были опробованы модели линейной регрессии, логистической регрессии и случайного леса;   
- Модель линейной регрессии обучалась 14-15 минут, при этом имела проблемы с предсказанием целевых значений;   
- Модель логистической регрессии обучается не более 10 секунд, с помощью этой модели была достигнута необходимая метрика как на валидационной, так и на тестовой выборках. Данная мадель предлагается в качестве конечной для поставленной задачи;   
- Модель случайного леса обучается более 3 часов на дефолтных гиперпараметрах, при этом контролируемая метрика значительно ниже (f1_score = 0.71) чем для дефолтных гиперпараметров логистической регрессии. В связи с этим, в частности неоправданными временными затратами, было принято решение от этой модели отказаться.

[*к содержанию*](#back)
___